In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING messages
import argparse
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from sentence_transformers import SentenceTransformer

## train test split
from sklearn.model_selection import train_test_split


/home/dimitri/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Parameters
n_missing = 3
MODEL_PATH = "rnn_model.h5"
TOKENIZER_PATH = "tokenizer.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 128

In [7]:
df = pd.read_csv("prompts_dataset.csv")

In [8]:
data = df.Prompt
note = df.Complexite

In [13]:
vocab_size = len(tokenizer.word_index) + 1

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM

# Charger le tokenizer et le modèle BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_model = AutoModel.from_pretrained("bert-base-multilingual-cased")

In [59]:
# Exemple de données
texts = list(data)[:1000]

# Tokenisation et encodage des textes
encoded_inputs = tokenizer(texts, padding=True, truncation=True, max_length=MAX_SEQUENCE_LENGTH, return_tensors="pt")

# Obtenir les embeddings de BERT
with torch.no_grad():
    outputs = bert_model(**encoded_inputs)
    # Utiliser les embeddings de la dernière couche cachée
    bert_embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

# Convertir les embeddings en numpy pour Keras
bert_embeddings = bert_embeddings.numpy()

# Entraîner le modèle (avec des données d'entraînement adaptées)
# model.fit(bert_embeddings_train, y_train, validation_data=(bert_embeddings_test, y_test), epochs=10, batch_size=32)

In [52]:
bert_embeddings.shape

(500, 68, 768)

In [60]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings, np.array(list(note)[:1000]), test_size=0.2, random_state=42)

In [61]:
# Construire le modèle LSTM avec les embeddings de BERT
model = Sequential()

# Ajouter des couches bidirectionnelles LSTM
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(MAX_SEQUENCE_LENGTH, bert_embeddings.shape[-1]))))
model.add(Bidirectional(LSTM(64)))

# Ajouter des couches denses
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compiler le modèle
model.compile(loss='mean_absolute_error', optimizer='adam')

In [17]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [62]:
# Train and evaluate model with train and test
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
25/25 [==============================] - 7s 142ms/step - loss: 1.1105 - val_loss: 0.8795
Epoch 2/10
25/25 [==============================] - 2s 96ms/step - loss: 0.8106 - val_loss: 0.7984
Epoch 3/10
25/25 [==============================] - 2s 96ms/step - loss: 0.6988 - val_loss: 0.7998
Epoch 4/10
25/25 [==============================] - 2s 100ms/step - loss: 0.6396 - val_loss: 0.7282
Epoch 5/10
25/25 [==============================] - 2s 100ms/step - loss: 0.6274 - val_loss: 0.7271
Epoch 6/10
25/25 [==============================] - 3s 102ms/step - loss: 0.5749 - val_loss: 0.7361
Epoch 7/10
25/25 [==============================] - 3s 102ms/step - loss: 0.5503 - val_loss: 0.7443
Epoch 8/10
25/25 [==============================] - 3s 106ms/step - loss: 0.5021 - val_loss: 0.7472
Epoch 9/10
25/25 [==============================] - 3s 107ms/step - loss: 0.4449 - val_loss: 0.7721
Epoch 10/10
25/25 [==============================] - 3s 108ms/step - loss: 0.4464 - val_loss: 0.7189


In [63]:
def predict(data):
    data = [data]
    # Tokenisation et encodage des textes
    encoded_inputs = tokenizer(data, padding=True, truncation=True, max_length=MAX_SEQUENCE_LENGTH, return_tensors="pt")
    # Obtenir les embeddings de BERT
    with torch.no_grad():
        outputs = bert_model(**encoded_inputs)
        # Utiliser les embeddings de la dernière couche cachée
        bert_embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

    # Convertir les embeddings en numpy pour Keras
    bert_embeddings = bert_embeddings.numpy()
    print(bert_embeddings.shape)
    prediction = model.predict(bert_embeddings)
    return prediction

In [69]:
predict("Un éléphant pèse 5 000 kg et tient sur une patte. Quel est le poids supporté par cette patte ?")

(1, 30, 768)
1/1 [==============================] - 0s 15ms/step


array([[1.9818515]], dtype=float32)